In [ ]:
import os
import cv2
import time
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset
from torch.utils.data import RandomSampler

import torchvision.transforms as T
import torchvision.models as models
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from xml.etree import ElementTree as et
#from torch_snippets import *

from IPython import display 
display.set_matplotlib_formats('svg')

from PIL import Image
from PIL import ImageFont, ImageDraw

from tqdm import tqdm

from torchvision.ops import box_iou
import albumentations as A


In [ ]:
train_data = pd.read_csv("../input/dbf-omsk-dataset/train_dataset_train/train.csv")
train_data_plus = train_data[train_data.count_region > 0]
train_data_plus

In [ ]:
#train_data

In [ ]:
import ast

def parse_region_shape(region_shape_str, image_id=0):
    data = ast.literal_eval(region_shape_str)
    
    list_with_all_boxes = []
    
    for box in data:
        box_dict = ast.literal_eval(box)
        #print(box_dict["cx"])
        # The COCO bounding box format is [top left x position, top left y position, width, height]
        coco_xmin = box_dict["cx"] - box_dict["r"] 
        coco_ymin = box_dict["cy"] - box_dict["r"] 
        coco_width = 2 * box_dict["r"] 
        coco_height = 2 * box_dict["r"] 
    
        coco_list_with_single_boxes = [coco_xmin, coco_ymin, coco_width, coco_height]
    
        line = {'category_id': 1.0, 'bbox': coco_list_with_single_boxes, 'area': 1.0}
        list_with_all_boxes.append(line)


    return list_with_all_boxes

region_shape_sample = train_data_plus.region_shape.values[5]
parse_region_shape(region_shape_sample)

In [ ]:
'''
transform = A.Compose([
    A.RandomCrop(width=450, height=450),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
], bbox_params=A.BboxParams(format='coco'))

transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
transformed_image = transformed['image']
transformed_bboxes = transformed['bboxes']

'''

In [ ]:
class CustomDataset(Dataset): 
    def __init__(self, file_dir, file_list, box_str_list = None, feature_extractor=None): 
        self.file_dir = file_dir
        self.file_list = file_list
        self.box_list = box_str_list
        self.feature_extractor = feature_extractor
        
    def __len__(self): 
        return len(self.file_list)
    
    def __getitem__(self, index): 
        
        #if self.file_list[index] == "2050.DS_Store":
        #    image_path = "../input/dbf-omsk-dataset/test_dataset_test/test/0.JPG"
        #else:
        image_path = self.file_dir + self.file_list[index]  
        box_str = self.box_list[index]  
        
        image_res = Image.open(image_path).convert('RGB')#.resize(self.width, self.height)        
        
        #wt = image_res.shape[1]
        #ht = image_res.shape[0]     
        
        target = {}
        
        target['annotations'] = parse_region_shape(box_str)

        target['image_id'] = index 
        
        if self.feature_extractor:
            return feature_extractor(images=image_res, annotations=target, return_tensors="pt")
        else:
            return image_res, target

In [ ]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("hustvl/yolos-base", size=512, max_size=768)

In [ ]:
test_df = pd.read_csv("../input/dbf-omsk-test2/sample_solution.csv")
test_df = test_df.drop(["count_region"], axis = 1)
test_df.ID_img.values

In [ ]:
file_dir_train = '../input/dbf-omsk-dataset/train_dataset_train/train/'
file_dir_test = '../input/dbf-omsk-test2/test_dataset_test/test/'

files_test = list(test_df.ID_img.values)
#os.listdir(file_dir_test) 
files_train = train_data_plus["ID_img"].values

test_boxed_dimmy = ["[ ]" for i in files_test]

In [ ]:

# Getting the dataset 
train_set = CustomDataset(file_dir_train, files_train, train_data_plus.region_shape.values, feature_extractor=feature_extractor)
test_set = CustomDataset(file_dir_test, files_test, test_boxed_dimmy, feature_extractor=feature_extractor)

print(len(train_set), len(test_set))


# Data Loader 
train_dataloader = DataLoader(train_set, batch_size = 1, shuffle = True, num_workers = 1)
test_loader = DataLoader(test_set, batch_size = 1, shuffle = False, num_workers = 1)

In [ ]:
tensor = train_set[9]['pixel_values'][0]
boxes = train_set[9]['labels'][0]['boxes']

tensor = 0.225 * tensor + 0.456

img = T.ToPILImage()(tensor)

print(img.size)
plt.figure(figsize=(20, 20))

draw = ImageDraw.Draw(img)
for box in boxes:
    print(box)
    x1, y1, w_size, h_size = box[0], box[1], box[2], box[3] 
    x_start = (x1 - (w_size/2)) * img.size[0]
    y_start = (y1 - (h_size/2)) * img.size[1]
    x_end = (x_start + w_size * img.size[0]) 
    y_end = (y_start + h_size * img.size[1]) 
    print(x_start, y_start, x_end, y_end)
    #draw.rectangle(((x_start, y_start), (x_end, y_end)), outline ="red")
    draw.ellipse(((x_start, y_start), (x_end, y_end)), outline ="red")
    
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
from transformers import DetrConfig, AutoModelForObjectDetection

#model = Detr(lr=2.5e-5, weight_decay=1e-4)
model = AutoModelForObjectDetection.from_pretrained("hustvl/yolos-base", num_labels=3, ignore_mismatched_sizes=True)
#'hustvl/yolos-base'  "hustvl/yolos-small"
lr=2.5e-5
weight_decay=1e-4

optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

In [ ]:
if torch.cuda.is_available():  
    model.cuda()

In [ ]:
model.load_state_dict(torch.load(f"../input/dbfmodelkrasnodar-v2/model_krasnodar.pth"), strict=False)

In [ ]:
train_set[7]['labels'][0]['boxes']

In [ ]:
def yolo_box2xy(box):
    
        x1, y1, w_size, h_size = box[:,0], box[:,1], box[:,2], box[:,3] 
        
        # для рисования
        x_start = (x1 - (w_size/2))
        y_start = (y1 - (h_size/2)) 
        x_end = (x_start + w_size ) 
        y_end = (y_start + h_size )
        #print(x_start)
        xy = torch.stack((x_start, y_start, x_end, y_end)).permute(1,0)
        return xy
        
yolo_box2xy(train_set[7]['labels'][0]['boxes'])

In [ ]:
for epoch_num in range(0):
    train_loss = 0
    train_loss_ce = 0
    train_loss_bbox = 0
    valid_loss = 0
    valid_loss_ce = 0
    valid_loss_bbox = 0
    
    valid_box_iou = 0
    loss = 0
    
    # Train loop   
    model.train()
    #print("Train step")
    for ii, batch in enumerate(iter(train_dataloader)):
        #print(batch["labels"])
        pixel_values = batch['pixel_values'][0]
        
        if torch.cuda.is_available():
            pixel_values = pixel_values.cuda()
            

        if torch.cuda.is_available():  
            labels = [{k: v[0].cuda() for k, v in t.items()} for t in batch['labels']]
        else:
            labels = [{k: v[0] for k, v in t.items()} for t in batch['labels']]
        
        #print(labels)
        
        outputs = model(pixel_values=pixel_values, labels=labels)
        
        #print(outputs)

        loss += outputs.loss
        if (ii+1)%4 == 0:       
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss = 0
        
        #train_loss += loss.item()
        train_loss_ce += outputs.loss_dict['loss_ce'].item()
        train_loss_bbox += outputs.loss_dict['loss_bbox'].item()
        
 
        boxes1 = yolo_box2xy(labels[0]['boxes'].cpu().detach())
        boxes2 = yolo_box2xy(outputs.pred_boxes[0].cpu().detach())

        valid_box_iou += torch.sum(box_iou(boxes1, boxes2))

        
        #break
    #break
        #if ii%10 == 0:
        #    print("Iter {}, Loss = {:.4f} {:.4f} {:.4f}".format(ii, loss.item(), outputs.loss_dict['loss_ce'].item(), outputs.loss_dict['loss_bbox'].item()))

    # Valid loop 
    '''
    model.eval()
    #print("Eval step")
    for ii, batch in enumerate(iter(val_dataloader)):
        
        if torch.cuda.is_available():
            pixel_values = pixel_values.cuda()
        
        if torch.cuda.is_available():  
            labels = [{k: v[0].cuda() for k, v in t.items()} for t in batch['labels']]
        else:
            labels = [{k: v[0] for k, v in t.items()} for t in batch['labels']]
        
        outputs = model(pixel_values=pixel_values, labels=labels)

        #loss = outputs.loss
        
        #valid_loss += loss.item()
        valid_loss_ce += outputs.loss_dict['loss_ce'].item()
        valid_loss_bbox += outputs.loss_dict['loss_bbox'].item()
    '''
    #valid_box_iou = valid_box_iou / 
    print("Epoch {}, Train loss CE={:.4f}, BBOX={:.4f}, IoU={:.2f}".format(epoch_num+1, train_loss_ce, train_loss_bbox, valid_box_iou))
    #print("Epoch {}, Valid loss CE={:.4f}, BBOX={:.4f}".format(epoch_num+1, valid_loss, valid_loss_ce, valid_loss_bbox))

In [ ]:
#torch.save(model.state_dict(), f"model_krasnodar.pth")


In [ ]:
def test_pred(i, set, thr=0.8):
    batch = set[i]
    pixel_values = batch['pixel_values']#[0]
    outputs = model(pixel_values=pixel_values.cuda())

    #outputs
    print(batch['labels'])

    outputs_softmax = torch.softmax(outputs.logits.detach(), dim=-1)[0, :, :-1]
    probs = torch.max(outputs_softmax, dim=-1).values.cpu()
    pred_class = torch.max(outputs_softmax, dim=-1).indices.cpu()

    tensor = pixel_values[0].cpu()
    boxes = outputs.pred_boxes[0].cpu().detach().numpy()

    tensor = 0.225 * tensor + 0.456

    img = T.ToPILImage()(tensor)

    print(img.size)
    plt.figure(figsize=(15, 15))

    draw = ImageDraw.Draw(img)
    for n, box in enumerate(boxes):
        #print(box)
        x1, y1, w_size, h_size = box[0], box[1], box[2], box[3] 
        x_start = (x1 - (w_size/2)) * img.size[0]
        y_start = (y1 - (h_size/2)) * img.size[1]
        x_end = (x_start + w_size * img.size[0]) 
        y_end = (y_start + h_size * img.size[1]) 

        if probs[n]>thr:
            print(n, int(x_start), int(y_start), int(x_end), int(y_end), probs[n].numpy(), pred_class[n].numpy())
            draw.ellipse(((x_start, y_start), (x_end, y_end)), outline ="red")

    plt.imshow(img)
    plt.axis('off')
    plt.show()
    
test_pred(9, train_set)

In [ ]:
test_pred(0, train_set)

In [ ]:
test_pred(5, train_set)

In [ ]:
test_pred(12, train_set)

# Test

In [ ]:
model.eval()
predicts = []

thr = 0.85

for i, batch in enumerate(test_set):
    #i = files_test.index("1275.JPG")
    #batch = test_set[i]
    
    #if files_test[i] == "2050.DS_Store":
    #    img_path = "../input/dbf-omsk-dataset/test_dataset_test/test/0.JPG"
    #else:
    #    
    img_path = file_dir_test + files_test[i]
    img = Image.open(img_path)
    
    if img.size[1] < 1000:
        predicts.append(0)
        continue
    
    pixel_values = batch['pixel_values']#[0]
    outputs = model(pixel_values=pixel_values.cuda())

    #outputs
    #print(batch['labels'])

    outputs_softmax = torch.softmax(outputs.logits.detach(), dim=-1)[0, :, :-1]
    probs = torch.max(outputs_softmax, dim=-1).values.cpu()
    pred_class = torch.max(outputs_softmax, dim=-1).indices.cpu()

    #tensor = pixel_values[0].cpu()
    boxes = outputs.pred_boxes[0].cpu().detach().numpy()

    #tensor = 0.225 * tensor + 0.456

    #img = T.ToPILImage()(tensor)


    #print(img.size)
    

    draw = ImageDraw.Draw(img)
    
    pred_lines = []
    pred_str = ""
    for n, box in enumerate(boxes):
        #print(box)
        x1, y1, w_size, h_size = box[0], box[1], box[2], box[3] 
        
        # для рисования
        x_start = (x1 - (w_size/2)) * img.size[0]
        y_start = (y1 - (h_size/2)) * img.size[1]
        x_end = (x_start + w_size * img.size[0]) 
        y_end = (y_start + h_size * img.size[1]) 
        
        # для предикта
        cx = int(x1 * img.size[0])
        cy = int(y1 * img.size[1])
        r = min(int((w_size / 2) * img.size[0]), int((h_size / 2) * img.size[1]))
        
        if r > cy: r = cy - 1
        if r > cx: r = cx - 1    
        if r > img.size[0] - cx: r = img.size[0] - cx - 1 
        if r > img.size[1] - cy: r = img.size[1] - cy - 1 
        
        # Убираем предикты с малым радиусом
        if (r < 30):
            continue
        
        # берем предикты больше порога
        if probs[n]>thr:
            #print(n, int(x_start), int(y_start), int(x_end), int(y_end), probs[n].numpy(), pred_class[n].numpy())
            pred_line = '{"cx":'+str(cx)+ ', "cy":'+ str(cy) + ', "r":' + str(r)+ '}'
            #pred_line = '\'{"cx":'+str(0)+ ', "cy":'+ str(0) + ', "r":' + str(0)+ "}\', "
            #print(pred_line)
            pred_lines.append(pred_line)
            draw.ellipse(((x_start, y_start), (x_end, y_end)), outline ="red", width=5)
            
        #if(len(pred_lines) == 5): break

    if len(pred_lines) == 0:
        predicts.append(0)
    else:
        #pred_str = "["
        #for item in pred_lines:
        #    pred_str += item
        #pred_str = pred_str[:-2]
        #pred_str += "]"
        #print(pred_str)
        predicts.append(pred_lines)
        
        plt.figure(figsize=(10, 10))
        plt.imshow(img)
        plt.axis('off')
        plt.show()
    
    print(i, files_test[i], img.size[0], img.size[1], len(pred_lines), pred_lines)

    



In [ ]:
len(predicts), test_df.shape[0]
#predicts[3061]
#predicts.index(['{"cx":3473, "cy":2322, "r":79}', '{"cx":3378, "cy":2308, "r":92}', '{"cx":3372, "cy":2194, "r":91}'])

In [ ]:
test_df["region_shape"] = predicts #[predicts[3061] for k in range(test_df.shape[0])]

len(test_df[test_df.region_shape != 0].index)

In [ ]:
#predicts[3383]

In [ ]:
#for idx in test_df[test_df.region_shape != 0].index:
#for idx in test_df.index:
#    test_df.loc[idx, "region_shape"] = ['{"cx":300, "cy":300, "r":100}' for x in range(2)]

In [ ]:
test_df[test_df.region_shape != 0]

In [ ]:
test_df.to_csv("submit80.csv", index=False)